In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np


# Ring Modulation

In [5]:
x, sr = librosa.load("../audio/suzanne.wav") 


# carrir oscillator
fc = 300

x_m = np.sin(2*np.pi*fc*np.arange(len(x))/sr);

# ring modulation
y = x*x_m;

ipd.Audio(y, rate=sr)



# FM Synthesis: Basic Tone


In [10]:
sr = 22050         # sampling rate 
fc = 440           # carrier frequency
fm = 220           # modulation frequency 
note_dur = 1       # length in second
beta = 10          # beta parameters in FM
attack_time = 0.01 # attack time in second

# amplitude envelope
env_attack = np.linspace(0,1,int(attack_time*sr))
env_decay = np.logspace(np.log10(1),np.log10(0.1), note_dur*sr-len(env_attack))
env = np.append(env_attack, env_decay)

# generate FM oscillator
t = np.linspace(0, note_dur, note_dur*sr)
x_note = np.sin(2*np.pi*fc*t + beta*np.sin(2*np.pi*fm*t))

# apply the ampltidue envelope
x_note_env = np.multiply(x_note, env)
    
ipd.Audio(x_note_env, rate=sr)

# FM Synthesis: Bell Sound

In [11]:
sr = 22050         # sampling rate 
fc = 200           # carrier frequency
fm = 280           # modulation frequency 
note_dur = 3       # length in second
beta_max = 10      # beta parameters in FM
attack_time = 0.01 # attack time in second


# amplitude envelope
env_attack = np.linspace(0,1,int(attack_time*sr))
env_decay = np.logspace(np.log10(1),np.log10(0.1), note_dur*sr-len(env_attack))
env = np.append(env_attack, env_decay)

# beta
beta = beta_max*env

# FM oscillator
t = np.linspace(0,note_dur, note_dur*sr)
x_note = np.sin(2*np.pi*fc*t + beta*np.sin(2*np.pi*fm*t));

# apply the ampltidue envelope
x_note_env = np.multiply(x_note, env)
    
ipd.Audio(x_note_env, rate=sr)

# FM Synthesis: Brass Instrument

In [12]:
sr = 22050         # sampling rate 
note_dur = 2       # length in second
beta_max = 5       # beta parameters in FM
attack_time = 0.1  # attack time in second

# MIDI note number: C4=60, C5=72, ... 
notes = [60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60]
note_legnths = [1,1,1,1,1,1,2,1,1,1,1,1,1,2]

# envelope
attack_time = 0.01 # second

x = []
for i in range(len(notes)):    
    # fundamental frequency
    f0= 440*np.power(2.0,(notes[i]-69)/12)
    note_dur = note_legnths[i]*0.3

    # amplitude envelope
    env_attack = np.linspace(0,1,int(attack_time*sr))
    env_decay = np.logspace(np.log10(1),np.log10(0.1), int(note_dur*sr)-len(env_attack))
    env = np.append(env_attack, env_decay)

    # beta
    beta = beta_max*env

    # FM oscillator
    fc = f0
    fm = f0
    t = np.linspace(0, note_dur, int(note_dur*sr))
    x_note = np.sin(2*np.pi*fc*t + beta*np.sin(2*np.pi*fm*t));

    # apply the ampltidue envelope
    x_note_env = np.multiply(x_note, env)
    
    # concatenate notes
    x = np.append(x, x_note_env)
    
ipd.Audio(x, rate=sr)



# FM Synthesis: Electric Piano

In [13]:
sr = 22050         # sampling rate 
note_dur = 2       # length in second
beta_max = 3       # beta parameters in FM
attack_time = 0.05  # attack time in second

# MIDI note number: C4=60, C5=72, ... 
notes = [60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60]
note_legnths = [1,1,1,1,1,1,2,1,1,1,1,1,1,2]

# envelope
attack_time = 0.01 # second

x = []
for i in range(len(notes)):    
    # fundamental frequency
    f0= 440*np.power(2.0,(notes[i]-69)/12)
    note_dur = note_legnths[i]*0.5

    # amplitude envelope
    env_attack = np.linspace(0,1,int(attack_time*sr))
    env_decay = np.logspace(np.log10(1),np.log10(0.1), int(note_dur*sr)-len(env_attack))
    env = np.append(env_attack, env_decay)

    # beta
    beta = beta_max*env

    # FM oscillator
    fc = f0
    fm = 10*fc
    t = np.linspace(0, note_dur, int(note_dur*sr))
    x_note = np.sin(2*np.pi*fc*t + beta*np.sin(2*np.pi*fm*t));

    # apply the ampltidue envelope
    x_note_env = np.multiply(x_note, env)
    
    # concatenate notes
    x = np.append(x, x_note_env)
    
ipd.Audio(x, rate=sr)



# Let's add Chorus 

In [14]:
def chorus(x, delay_time, lfo_rate, lfo_depth, sr):

# A simple chorus effect
# 
# - x: input signal
# - delay_time (in second): delay time
# - lfo_depth (time): control the excursion of modulation
# - lfo_rate  (Hz): control the rate of modulation
# - sr: sampling rate


    y = np.zeros(len(x))

    # length of delayline
    MAX_DELAY_LENGTH = sr*2

    # write pointer
    wp = 0

    # length of delayline
    delay_length = delay_time*sr;

    # fill delayline with random noise
    delayline = np.zeros(MAX_DELAY_LENGTH);
    
    # initialize LFO settings
    lfo_depth_samples = lfo_depth*sr
    lfo_phase = 0
    lfo_phase_inc  = 2*np.pi*lfo_rate/sr;


    # feed-forward loop
    for n in range(len(x)):

        # LFO
        lfo_phase = lfo_phase + lfo_phase_inc
        if lfo_phase > 2*np.pi:
            lfo_phase = lfo_phase - 2*np.pi

        lfo_out = lfo_depth_samples*np.sin(lfo_phase)
        
        # read sample
        op = wp - delay_length
        op = op + lfo_out

        if ( op < 0.00 ):
            op = op + MAX_DELAY_LENGTH

        rp = np.floor(op)
        rp_frac = op - rp

        rp = int(rp)
        
        # read sample using linear intepolation        
        if (rp == (MAX_DELAY_LENGTH-1) | (rp == -1) ):
            tap_out = (1-rp_frac)*delayline[MAX_DELAY_LENGTH-1] + rp_frac*delayline[0]
        else:
            tap_out = (1-rp_frac)*delayline[rp-1] + rp_frac*delayline[rp]

        # write output
        delayline[wp] = x[n]

        # update write pointer
        wp = wp + 1
        if wp >= MAX_DELAY_LENGTH:
            wp = 0

#        print(tap_out, x[n])    
            
        y[n] = x[n] + tap_out
        
    return y



In [15]:
lfo_rate = 2 # Hz
lfo_depth = 0.0005 #

delay_time = 0.02

x = np.append(x, np.zeros(1*sr))

y_chorus = chorus(x, delay_time, lfo_rate, lfo_depth, sr)

ipd.Audio(y_chorus, rate=sr)

# Let's Add Reverb 

In [16]:
h_reverb, sr = librosa.load("../audio/memchu_ir.wav") 
print(sr)

y_reverb = np.convolve(y_chorus, h_reverb)

ipd.Audio(y_reverb, rate=sr)


22050
